In [2]:
using HomotopyContinuation, DynamicPolynomials
using Plots

In [3]:
#generates a random number between -n and n
function genrand(n)
    return rand()*n*2 - n
end

@polyvar x y
#creates a random polynomial with coeffcients between -n and n
function get_random_polynomial(n)
    p = MultivariatePolynomials.polynomial([genrand(n), genrand(n), genrand(n), genrand(n), genrand(n), genrand(n)], [x^2, x, x*y, y, y^2, 1])
    return p
end

#takes the norm of a polynomial by squarring the coefficients
function get_norm(p)
    normop = norm(MultivariatePolynomials.coefficients(p, [x^2, x, x*y, y, y^2, 1]))
    return normop
end

get_norm (generic function with 1 method)

## This notebook compares the polynomial simplification function by either using the groebner basis or by using a function to minimise
### The function below uses groebner basis to find the coefficients then the minimise function to find the a1,...,b3 
This is the one that is not able to find a solution, from the test below.

In [4]:
function groebn_then_mini(p)
    
    #Put in u0 the coefs we have for our point
    u₀ = MultivariatePolynomials.coefficients(p, [x^2, x, x*y, y, y^2, 1])
    
    #Using HomotopyContinuation to find optimal solution
    @var c1 c2 c3 c4 c5 c6
    @var u[1:6] λ[1:1]
    f = c1*c4^2 - 4*c1*c5*c6 + c2^2*c5 - c2*c3*c4 + c3^2*c6

    J = differentiate([f], [c1,c2,c3,c4,c5,c6])
    C = System([[c1,c2,c3,c4,c5,c6] - u - J'*λ; f], variables = [c1;c2;c3;c4;c5;c6;λ], parameters = u)
    #We get the solutions
    solution = HomotopyContinuation.solve(C; target_parameters = u₀, show_progress= false)
    real_sols = real_solutions(solution; tol=1e-5)
    #stops if it cant find real solutions anymore to avoid an error
    if real_sols == []
        return false, 0
    end
    
    #We take only the real solutions
    ed_points = map(p -> p[1:6], real_sols)

    
    #find optimal c1,...,c6 
    _, idx = findmin([norm(x - u₀) for x in ed_points])  
    c1d,c2d,c3d,c4d,c5d,c6d = ed_points[idx]
    
    #use our minimise function to find the a1...b3
    @var a1 a2 a3 b1 b2 b3
    f = (c1d - a1*b1)^2 + (c2d - a1*b3 - a3*b1)^2 + (c3d - a1*b2 - a2*b1)^2 + (c4d - a2*b3 - a3*b2)^2 + (c5d - a2*b2)^2 + (c6d - a3*b3)^2
    
    #find the gradient
    J = differentiate(f, [a1,a2,a3,b1,b2,b3])
    #push in J the linear function
    eq = genrand(5)*a1 + genrand(5)*a2 + genrand(5)*a3 + genrand(5)*b1 + genrand(5)*b2 + genrand(5)*b3 + genrand(5)
    push!(J, eq)
    
    #set the system and find the results
    system = System(J; variables = [a1,a2,a3,b1,b2,b3])
    
    result = HomotopyContinuation.solve(system; show_progress= false)

    real_sols = real_solutions(result; tol=1e-5)

    
    function mf(v)
        a1, a2, a3, b1, b2, b3 = v
        return (c1d - a1*b1)^2 + (c2d - a1*b3 - a3*b1)^2 + (c3d - a1*b2 - a2*b1)^2 + (c4d - a2*b3 - a3*b2)^2 + (c5d - a2*b2)^2 + (c6d - a3*b3)^2
    end
    #get the minimum solution and output
    minval, minindex = findmin(map(s -> mf(s[1:6]), real_sols))

    a = real_sols[minindex][1:6]
    
    alpha = (a[1] * x + a[2] * y + a[3])
    beta = (a[3] * x + a[4] * y + a[5])
    
    return alpha, beta
    
end

getalphabetatry (generic function with 1 method)

Function below takes a random polynomial then loops it and every loop it ouputs: <br>
i : norm <br>
Where i is the number of the loop and norm is the norm of the polynomial at that loop number. <br>
From the results there we can see that the norm grows instead of diminishing.

In [5]:
p = get_random_polynomial(10)
for i in 1:10
    a,b=groebn_then_mini(p)
    p = p - (a*b)
    println(i, ": ", get_norm(p))
end

Computing mixed cells... 14 	 Time: 0:00:00
  mixed_volume:  90


1: 31.533274588253832
2: 169.12640097105103
3: 169.35656222549602
4: 171.4808346946532
5: 171.3602829342806
6: 294.6513429063386
7: 297.6979402151545
8: 519.2594325217102
9: 1622.3721304718224
10: 3338.0024211093837


### The function below uses the minimise function but instead of setting a1=1 we add the linear function of the a1,...,b3.
This function probably has the same problem as the one above

In [9]:
function groebner(p,n)    
    
    c = MultivariatePolynomials.coefficients(p, [x^2, x, x*y, y, y^2, 1])
    
    @var a1 a2 a3 b1 b2 b3
    f = (c[1] - a1*b1)^2 + (c[2] - a1*b3 - a3*b1)^2 + (c[3] - a1*b2 - a2*b1)^2 + (c[4] - a2*b3 - a3*b2)^2 + (c[5] - a2*b2)^2 + (c[6] - a3*b3)^2
    
    #get the gradient
    J = differentiate(f, [a1,a2,a3,b1,b2,b3])
    #push in J the linear function
    eq = genrand(n)*a1 + genrand(n)*a2 + genrand(n)*a3 + genrand(n)*b1 + genrand(n)*b2 + genrand(n)*b3 + genrand(n)
    push!(J, eq)
    
    #set the system and find the results
    system = System(J; variables = [a1,a2,a3,b1,b2,b3])

    result = solve(system; show_progress= false)

    real_sols = real_solutions(result; tol=1e-5)

    #stops if it cant find real solutions anymore to avoid an error
    if real_sols == []
        return false, 0
    end
    
    function mf(v)
        a1, a2, a3, b1, b2, b3 = v
        return (c[1] - a1*b1)^2 + (c[2] - a1*b3 - a3*b1)^2 + (c[3] - a1*b2 - a2*b1)^2 + (c[4] - a2*b3 - a3*b2)^2 + (c[5] - a2*b2)^2 + (c[6] - a3*b3)^2
    end
    
    #we find the minimum and output the answer
    _, minindex = findmin(map(s -> mf(s[1:6]), real_sols))
    

    a = real_sols[minindex][1:6]

    alpha = (a[1] * x + a[2] * y + a[3])
    beta = (a[3] * x + a[4] * y + a[5])
    
    return alpha, beta
end

groebner (generic function with 1 method)

Function below uses same principal as one up top, to see the progression of the norms as the function loops. <br>
Does not seem to work as well.

In [8]:
p = get_random_polynomial(10)
for i in 1:10
    a,b=groebner(p,10)
    p = p - (a*b)
    println(i, ": ", get_norm(p))
end

1: 8.16579323631429
2: 12.228605048245754
3: 20.51023309294352
4: 20.769797449059176
5: 25.75191646126316
6: 97.44109604380992
7: 110.83603233752828
8: 543.1971381860117
9: 543.2463092156752
10: 1570.6308204790314


### The function below just uses the minimise function with a1=1 and then a2=1 and then outputs the best result

In [17]:
function just_minimise(p)    
    
    c = MultivariatePolynomials.coefficients(p, [x^2, x, x*y, y, y^2, 1])
    
    #for a1 = 1
    @var a1 a2 a3 b1 b2 b3
    f1 = (c[1] - b1)^2 + (c[2] - b3 - a3*b1)^2 + (c[3] - b2 - a2*b1)^2 + (c[4] - a2*b3 - a3*b2)^2 + (c[5] - a2*b2)^2 + (c[6] - a3*b3)^2
    
    #get the gradient
    J1 = differentiate(f1, [a2,a3,b1,b2,b3])
    system1 = System(J1; variables = [a2,a3,b1,b2,b3])

    result1 = solve(system1; show_progress= false)

    real_sols1 = real_solutions(result1; tol=1e-5)
    
    
    #for a2 = 1
    f2 = (c[1] - a1*b1)^2 + (c[2] - a1*b3 - a3*b1)^2 + (c[3] - a1*b2 - b1)^2 + (c[4] - b3 - a3*b2)^2 + (c[5] - b2)^2 + (c[6] - a3*b3)^2
    
    #get the gradient
    J2 = differentiate(f2, [a1,a3,b1,b2,b3])
    system2 = System(J2; variables = [a1,a3,b1,b2,b3])

    result2 = solve(system2; show_progress= false)

    real_sols2 = real_solutions(result2; tol=1e-5)
    
    #checks if we have any solution
    sol=[true,true]
    if real_sols1 == []
        sol[1] = false
    end
    if real_sols2 == []
        sol[2] = false
    end
    if ! sol[1] && ! sol[2]
        return false,0
    end
    
    
    function mf1(v)
        a2, a3, b1, b2, b3 = v
        return (c[1] - b1)^2 + (c[2] - b3 - a3*b1)^2 + (c[3] - b2 - a2*b1)^2 + (c[4] - a2*b3 - a3*b2)^2 + (c[5] - a2*b2)^2 + (c[6] - a3*b3)^2
    end
    
    function mf2(v)
        a1, a3, b1, b2, b3 = v
        return (c[1] - a1*b1)^2 + (c[2] - a1*b3 - a3*b1)^2 + (c[3] - a1*b2 - b1)^2 + (c[4] - b3 - a3*b2)^2 + (c[5] - b2)^2 + (c[6] - a3*b3)^2
    end
    
    if sol[1]
        minval1, minindex1 = findmin(map(s -> mf1(s[1:5]), real_sols1))
        minarg1 = real_sols1[minindex1][1:5]
    end
    if sol[2]
        minval2, minindex2 = findmin(map(t -> mf2(t[1:5]), real_sols2))
        minarg2 = real_sols2[minindex2][1:5]
    end
    
    #find the minimum value for the best solution
    if sol[1] && sol[2]
        if minval1 < minval2
            sol[2] = false
        end
        sol[1] = false
    end
    
    if sol[1]
        alpha = (x + minarg1[1] * y + minarg1[2])
        beta = (minarg1[3] * x + minarg1[4] * y + minarg1[5])
        return alpha, beta
    end
        
    alpha = (minarg2[1]*x + y + minarg2[2])
    beta = (minarg2[3] * x + minarg2[4] * y + minarg2[5])
    return alpha, beta
end

just_minimise (generic function with 1 method)

Function below uses same principal as one up top, to see the progression of the norms as the function loops. <br>
Does work very well.

In [18]:
p = get_random_polynomial(10)
for i in 1:10
    a,b=just_minimise(p)
    p = p - (a*b)
    println(i, ": ", get_norm(p))
end
#from we can see this way of doing it works using either a1=1 or a2=1 (using the minimise function)

1: 1.8957243771010666
2: 0.5232564169540154
3: 0.08081984855461148
4: 0.00444937584389439
5: 0.0010260717568722146
6: 6.412832002894489e-5
7: 1.3199998404992888e-6
8: 1.2567710936868342e-8
9: 1.5744331880570605e-9
10: 7.211617588799882e-12


### The function below uses groebner to find the coefficients and then minimise function with a1=1 and a2=1 to find the a1,...,b3 then outputs the best result
function is not really important

In [21]:
function ngroebner_mini(p)
    
    #Put in u0 the coefs we have for our point
    u₀ = MultivariatePolynomials.coefficients(p, [x^2, x, x*y, y, y^2, 1])
    
    #Using HomotopyContinuation to find optimal solution
    @var c1 c2 c3 c4 c5 c6
    @var u[1:6] λ[1:1]
    f = c1*c4^2 - 4*c1*c5*c6 + c2^2*c5 - c2*c3*c4 + c3^2*c6

    J = differentiate([f], [c1,c2,c3,c4,c5,c6])
    C = System([[c1,c2,c3,c4,c5,c6] - u - J'*λ; f], variables = [c1;c2;c3;c4;c5;c6;λ], parameters = u)
    #We get the solutions
    solution = HomotopyContinuation.solve(C; target_parameters = u₀, show_progress= false)
    real_sols = real_solutions(solution; tol=1e-5)
    #stops if it cant find real solutions anymore to avoid an error
    if real_sols == []
        return false, 0
    end
    
    #We take only the real solutions
    ed_points = map(p -> p[1:6], real_sols)

    #find optimal c1,...,c6 
    _, idx = findmin([norm(x - u₀) for x in ed_points])  
    c1d,c2d,c3d,c4d,c5d,c6d = ed_points[idx]
    
    #for a1 = 1
    @var a1 a2 a3 b1 b2 b3
    f1 = (c1d - b1)^2 + (c2d - b3 - a3*b1)^2 + (c3d - b2 - a2*b1)^2 + (c4d - a2*b3 - a3*b2)^2 + (c5d - a2*b2)^2 + (c6d - a3*b3)^2
    
    #get the gradient
    J1 = differentiate(f1, [a2,a3,b1,b2,b3])
    system1 = System(J1; variables = [a2,a3,b1,b2,b3])

    result1 = solve(system1; show_progress= false)

    real_sols1 = real_solutions(result1; tol=1e-5)
    
    #for a2 = 1
    f2 = (c1d - a1*b1)^2 + (c2d - a1*b3 - a3*b1)^2 + (c3d - a1*b2 - b1)^2 + (c4d - b3 - a3*b2)^2 + (c5d - b2)^2 + (c6d - a3*b3)^2
    
    #get the gradient
    J2 = differentiate(f2, [a1,a3,b1,b2,b3])
    system2 = System(J2; variables = [a1,a3,b1,b2,b3])

    result2 = solve(system2; show_progress= false)

    real_sols2 = real_solutions(result2; tol=1e-5)
    
    #checks if we have any solution
    sol=[true,true]
    if real_sols1 == []
        sol[1] = false
    end
    if real_sols2 == []
        sol[2] = false
    end
    if ! sol[1] && ! sol[2]
        return false,0
    end
    
    function mf1(v)
        a2, a3, b1, b2, b3 = v
        return (c1d - b1)^2 + (c2d - b3 - a3*b1)^2 + (c3d - b2 - a2*b1)^2 + (c4d - a2*b3 - a3*b2)^2 + (c5d - a2*b2)^2 + (c6d - a3*b3)^2
    end
    
    function mf2(v)
        a1, a3, b1, b2, b3 = v
        return (c1d - a1*b1)^2 + (c2d - a1*b3 - a3*b1)^2 + (c3d - a1*b2 - b1)^2 + (c4d - b3 - a3*b2)^2 + (c5d - b2)^2 + (c6d - a3*b3)^2
    end
    
    if sol[1]
        minval1, minindex1 = findmin(map(s -> mf1(s[1:5]), real_sols1))
        minarg1 = real_sols1[minindex1][1:5]
    end
    if sol[2]
        minval2, minindex2 = findmin(map(t -> mf2(t[1:5]), real_sols2))
        minarg2 = real_sols2[minindex2][1:5]
    end
    
    #find the minimum value for the best solution
    if sol[1] && sol[2]
        if minval1 < minval2
            sol[2] = false
        end
        sol[1] = false
    end
    
    if sol[1]
        alpha = (x + minarg1[1] * y + minarg1[2])
        beta = (minarg1[3] * x + minarg1[4] * y + minarg1[5])
        return alpha, beta
    end
        
    alpha = (minarg2[1]*x + y + minarg2[2])
    beta = (minarg2[3] * x + minarg2[4] * y + minarg2[5])
    return alpha, beta
    
end

ngroebner_mini (generic function with 1 method)

Works but not as well as just using the minimise function

In [22]:
p = get_random_polynomial(10)
for i in 1:10
    a,b=ngroebner_mini(p)
    p = p - (a*b)
    println(i, ": ", get_norm(p))
end
#this is using groebner basis and then instead of nlsolve we use either a1=1 or a2=1 it works but
#not as well as uing the minimise function istead of groebner

1: 2.9589122324590655
2: 0.17790183053776812
3: 0.04090239707713159
4: 0.028079300781755478
5: 0.005790027829793603
6: 0.001988539550744906
7: 0.00019105902695931273
8: 5.1368517052186636e-5
9: 2.2857300066868715e-6
10: 2.2857300066868715e-6


Function that outputs the time needed for a certain number of iterations and also the resulting norm

In [24]:
@polyvar x y
loops = 20
averagetime1 = 0
averagenorm1 = 0
s = [0,0]
for _ in 1:loops
    print("+")
    p = get_random_polynomial(1000)
    t = time()
    for _ in 1:20
        a,b=just_minimise(p) 
        if a == false
            s[1] += 1
            break
        end
        p = p - (a*b)  
    end
    averagetime1 += time() - t
    polnorm = get_norm(p)
    #=if polnorm > 1
        println("H: ",polnorm)
    end=#
    averagenorm1 += polnorm
end
averagetime1 = averagetime1/loops
averagenorm1 = averagenorm1/loops

averagetime2 = 0
averagenorm2 = 0
for _ in 1:loops
    print("+")
    c = get_random_polynomial(1000)
    t = time()
    for _ in 1:loops
        a,b=ngroebner_mini(c) 
        if a == false
            s[2] += 1
            break
        end
        c = c - (a*b)  
    end
    averagetime2 += time() - t
    opolnorm = get_norm(c)
    #=if opolnorm > 1
        println("O: ",opolnorm)
    end=#
    averagenorm2 += opolnorm
end
averagetime2 = averagetime2/loops
averagenorm2 = averagenorm2/loops

println("In ",loops, " iterations...")
println("Using minimise: ")
println("Time: ", averagetime1,"s", " - Norm: ", averagenorm1)
println("Using homotopycontinuation: ")
println("Time: ", averagetime2,"s", " - Norm: ", averagenorm2)
println(s)

++++++++++++++++++++++++++++++++++++++++In 20 iterations...
Using minimise: 
Time: 3.32385333776474s - Norm: 3.4332317923633006e-6
Using homotopycontinuation: 
Time: 2.772345685958862s - Norm: 7.924014699321273e-6
[12, 20]
